In [1]:
#@title Import dependencies and check whether GPU is available. { display-mode: "form" }
from transformers import T5EncoderModel, T5Tokenizer
import torch
import torch.nn.functional as F
import h5py
import time
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bar      
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {}".format(device))

/home/liwh/.conda/envs/dnabert/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda


In [5]:
def get_T5_model():
    model = T5EncoderModel.from_pretrained("prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained('prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    return model, tokenizer
model, tokenizer = get_T5_model()

In [2]:
df = pd.read_pickle("./data/processed_mutations.dataset")
# print(df['label'].value_counts())
df = df[(df['label'] == 0) | (df['label'] == 1) | (df['label'] == 3) | (df['label'] == 2)]
df = df[df['mut0'].str.len() <= 5000]
# df = df[df['mut0'].str.len() >= 512]
df['pos'] = df['Feature range(s)'].str[0].str.split('-', expand=True)[0].astype(int)
df = df[df['Feature range(s)'].str[0].str.split('-', expand=True)[0] == df['Feature range(s)'].str[0].str.split('-', expand=True)[1]]
# df = df[df['mut1'].str.len()-df['pos'] > 256]
# df = df[df['pos'] > 256]
df.reset_index(drop=True, inplace=True)
# df = df.head()

#Truncate the sequence length to 513
# for i in range(len(df)):
#     pos = int(df.loc[i, 'Feature range(s)'][0].split('-')[0])
#     df["mut0"][i] = df["mut0"][i][(pos - 257):(pos + 256)]
#     df["mut1"][i] = df["mut1"][i][(pos - 257):(pos + 256)]
# df = df[['par0', 'mut0', 'mut1', 'label']]
df

,#Feature AC,Feature range(s),Original sequence,Resulting sequence,Feature short label,Feature type,Feature annotation,Affected protein AC,Affected protein symbol,Affected protein full name,...,parAC,par0,mut1,label,mutAC1,mut0_51,mut1_51,mut0_1025,mut1_1025,pos
0,EBI-10039489,[81-81],[V],[E],P28795:p.Val81Glu,mutation disrupting(MI:0573),,P28795,PEX3,,...,Q03694,MVLSRGETKKNSVRLTAKQEKKPQSTFQTLKQSLKLSNNKKLKQDS...,MAPNQRSRSLLQRHRGKVLISLTGIAALFTTGSVVVFFVKRWLYKQ...,0,P28795_P28795:p.Val81Glu,IKEQIKRRFEQTQEDSLYTIYELLPVWRMVLNENDLNLDSIVTQLK...,IKEQIKRRFEQTQEDSLYTIYELLPEWRMVLNENDLNLDSIVTQLK...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,81
1,EBI-10039495,[188-188],[N],[I],P28795:p.Asn188Ile,mutation decreasing(MI:0119),,P28795,PEX3,,...,Q03694,MVLSRGETKKNSVRLTAKQEKKPQSTFQTLKQSLKLSNNKKLKQDS...,MAPNQRSRSLLQRHRGKVLISLTGIAALFTTGSVVVFFVKRWLYKQ...,1,P28795_P28795:p.Asn188Ile,NEYLDSAIKLTMQQENCNKLQNRFYNWVTSWWSDPEDKADDAMVMA...,NEYLDSAIKLTMQQENCNKLQNRFYIWVTSWWSDPEDKADDAMVMA...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,188
2,EBI-10039873,[146-146],[K],[R],O95863:p.Lys146Arg,mutation decreasing strength(MI:1133),,O95863,SNAI1,,...,Q09472,MAENVVEPGPPSAKRPKLSSPALSASASDGTDFGSLFDLEHDLPDE...,MPRSFLVRKPSDPNRKPNYSELQDSNPEFTFQQPYDQAHLLAAIPP...,1,O95863_O95863:p.Lys146Arg,LEAEAYAAFPGLGQVPKQLAQLSEAKDLQARKAFNCKYCNKEYLSL...,LEAEAYAAFPGLGQVPKQLAQLSEARDLQARKAFNCKYCNKEYLSL...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,146
3,EBI-10039883,[187-187],[K],[R],O95863:p.Lys187Arg,mutation decreasing strength(MI:1133),,O95863,SNAI1,,...,Q09472,MAENVVEPGPPSAKRPKLSSPALSASASDGTDFGSLFDLEHDLPDE...,MPRSFLVRKPSDPNRKPNYSELQDSNPEFTFQQPYDQAHLLAAIPP...,1,O95863_O95863:p.Lys187Arg,EYLSLGALKMHIRSHTLPCVCGTCGKAFSRPWLLQGHVRTHTGEKP...,EYLSLGALKMHIRSHTLPCVCGTCGRAFSRPWLLQGHVRTHTGEKP...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,187
4,EBI-10041181,[368-368],[W],[A],Q9UKV5:p.Trp368Ala,mutation decreasing(MI:0119),,Q9UKV5,AMFR,,...,P60604,MAGTALKRLMAEYKQLTLNPPEGIVAGPMNEENFFEWEALIMGPED...,MPLLFLERFPWPSLRTYTGLSGLALLGTIISAYRALSQPEAGPGEP...,1,Q9UKV5_Q9UKV5:p.Trp368Ala,ICWDSMQAARKLPCGHLFHNSCLRSWLEQDTSCPTCRMSLNIADNN...,ICWDSMQAARKLPCGHLFHNSCLRSALEQDTSCPTCRMSLNIADNN...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21116,EBI-9993467,[413-413],[S],[A],P41279:p.Ser413Ala,mutation decreasing(MI:0119),,P41279,MAP3K8,,...,Q13526,MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MEYMSTGSDNKEEIDLLIKHLNVSDVIDIMENLYASEEPAVYEPSL...,1,P41279_P41279:p.Ser413Ala,LNPPREDQPRCQSLDSALLERKRLLSRKELELPENIADSSCTGSTE...,LNPPREDQPRCQSLDSALLERKRLLARKELELPENIADSSCTGSTE...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,413
21117,EBI-9994728,[418-418],[D],[A],Q53EL6:p.Asp418Ala,mutation decreasing(MI:0119),,Q53EL6,PDCD4,,...,P60842,MSASQDSRSRDNGPDGMEPEGVIESNWNEIVDSFDDMNLSESLLRG...,MDVENEQILNVNPADPDNLSDSLFSGDEENAGTEEIKNEINGNWIS...,1,Q53EL6_Q53EL6:p.Asp418Ala,SSTITVDQMKRGYERIYNEIPDINLDVPHSYSVLERFVEECFQAGI...,SSTITVDQMKRGYERIYNEIPDINLAVPHSYSVLERFVEECFQAGI...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,418
21118,EBI-9998311,[300-300],[T],[A],Q676U5:p.Thr300Ala,mutation increasing rate(MI:1131),,Q676U5,ATG16L1,,...,P42574,MENTENSVDSKSIKNLEPKIIHGSESMDSGISLDNSYKMDYPEMGL...,MSSGLRAADFPRWKRHISEQLRRRDRLQRQAFEEIILQYNKLLEKS...,3,Q676U5_Q676U5:p.Thr300Ala,LLDSITNIFGRRSVSSFPVPQDNVDTHPGSGKEVRVPATALCVFDA...,LLDSITNIFGRRSVSSFPVPQDNVDAHPGSGKEVRVPATALCVFDA...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,300
21119,EBI-9998316,[299-299],[D],[E],Q676U5:p.Asp299Glu,mutation disrupting rate(MI:1129),,Q676U5,ATG16L1,,...,

In [7]:
def prottrans_embedding(sequence):
    seq = ' '.join(list(sequence))
    seq = [seq]
    # print(seq)
    ids = tokenizer.batch_encode_plus(seq, add_special_tokens=True, padding=True)
    # print(ids)
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)
    with torch.no_grad():
        embedding = model(input_ids=input_ids,attention_mask=attention_mask)
    embedding = embedding.last_hidden_state.cpu().numpy()
    features = [] 
    for seq_num in range(len(embedding)):
        seq_len = (attention_mask[seq_num] == 1).sum()
        seq_emd = embedding[seq_num][:seq_len-1]
        features.append(seq_emd)

    return features


In [45]:
# embedding_df = pd.DataFrame()
# embedding_df['mut0']= df['mut0'].apply(prottrans_embedding)
# embedding_df.to_csv('data/embeddings/prottrans_embedding.csv')
dataset = []
for i in tqdm(range(len(df))):
    pos = int(df.loc[i, 'Feature range(s)'][0].split('-')[0])
    mut0_emb = prottrans_embedding(df['mut0'][i])[0].astype(np.float32)
    padded_mut0 = F.pad(torch.from_numpy(mut0_emb), (0, 0, 256, 256), "constant", 0)
    mut0_emb = padded_mut0[(pos - 1):(pos + 512)]
    # print(mut0_emb.shape)
    mut1_emb = prottrans_embedding(df['mut1'][i])[0].astype(np.float32)
    padded_mut1 = F.pad(torch.from_numpy(mut1_emb), (0, 0, 256, 256), "constant", 0)
    mut1_emb = padded_mut1[(pos - 1):(pos + 512)]
    par0_emb = prottrans_embedding(df['par0'][i])[0].astype(np.float32)
    par0_emb = np.vstack((par0_emb, np.zeros((1025,1024)))).astype(np.float32)
    par0_emb = par0_emb[:1024]
    entry = (torch.tensor(mut0_emb), torch.tensor(mut1_emb), torch.tensor(par0_emb), torch.tensor(df['label'][i]))
    dataset.append(entry)
    
    # print(par0_emb.dtype)
torch.save(dataset, 'data/embeddings/prottrans_embedding_512_5000.pt')

  0%|          | 0/11960 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'to'

In [ ]:
def one_hot_encode_protein_sequence(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    one_hot_matrix = np.zeros((len(sequence), len(amino_acids)))
    
    # 填充One-Hot矩阵
    for i, aa in enumerate(sequence):
        if aa in aa_to_index:
            one_hot_matrix[i, aa_to_index[aa]] = 1
        else:
            raise ValueError(f"未知的氨基酸：{aa}")
    
    return one_hot_matrix

dataset = []
for i in tqdm(range(len(df))):
    mut0_emb = one_hot_encode_protein_sequence(df['mut0'][i]).astype(np.float32)
    mut1_emb = one_hot_encode_protein_sequence(df['mut1'][i]).astype(np.float32)
    par0_emb = one_hot_encode_protein_sequence(df['par0'][i]).astype(np.float32)
    par0_emb = np.vstack((par0_emb, np.zeros((1025,20)))).astype(np.float32)
    par0_emb = par0_emb[:1024]
    entry = (torch.tensor(mut0_emb), torch.tensor(mut1_emb), torch.tensor(par0_emb), torch.tensor(df['label'][i]))
    dataset.append(entry)
    
    # print(par0_emb.dtype,par0_emb.shape,par0_emb)
torch.save(dataset, 'data/embeddings/onehot_embedding.pt')

In [50]:
df = pd.read_pickle("./data/processed_mutations.dataset")
# print(df['label'].value_counts())
df = df[(df['label'] == 0) | (df['label'] == 1) | (df['label'] == 3) | (df['label'] == 2)]
df = df[df['mut0'].str.len() <= 5000]
# df = df[df['mut0'].str.len() >= 512]
df['pos'] = df['Feature range(s)'].str[0].str.split('-', expand=True)[0].astype(int)
# df = df[df['mut1'].str.len()-df['pos'] > 256]
# df = df[df['pos'] > 256]
df.reset_index(drop=True, inplace=True)
dataset = []
for i in tqdm(range(len(df))):
    pos = int(df.loc[i, 'Feature range(s)'][0].split('-')[0])
    # mut0_emb = prottrans_embedding(df['mut0'][i])[0].astype(np.float32)
    # padded_mut0 = F.pad(torch.from_numpy(mut0_emb), (0, 0, 256, 256), "constant", 0)
    # mut0_emb = padded_mut0[(pos - 1):(pos + 512)]
    # # print(mut0_emb.shape)
    # mut1_emb = prottrans_embedding(df['mut1'][i])[0].astype(np.float32)
    # padded_mut1 = F.pad(torch.from_numpy(mut1_emb), (0, 0, 256, 256), "constant", 0)
    # mut1_emb = padded_mut1[(pos - 1):(pos + 512)]
    # par0_emb = prottrans_embedding(df['par0'][i])[0].astype(np.float32)
    # par0_emb = np.vstack((par0_emb, np.zeros((1025,1024)))).astype(np.float32)
    # par0_emb = par0_emb[:1024]
    entry = (df['mut0'][i], df['mut1'][i], df['par0'][i], df['label'][i],pos)
    dataset.append(entry)
    
    # print(par0_emb.dtype)
# torch.save(dataset, 'data/embeddings/prottrans_embedding_512_5000.pt')

100%|██████████| 23280/23280 [00:00<00:00, 24665.64it/s]


In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor
from torchvision.datasets import CIFAR10
from torchvision.models import resnet50
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import torch.nn.functional as F
import os
from torchvision import transforms
import pandas as pd
import h5py
import math
from tqdm import tqdm
import matplotlib.pyplot as plt 
from math import sqrt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import numpy as np
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
for mut0, mut1, par0, labels, pos in train_loader:
    mut0_emb = prottrans_embedding(mut0)[0].astype(np.float32)
    padded_mut0 = F.pad(torch.from_numpy(mut0_emb), (0, 0, 256, 256), "constant", 0)
    mut0_emb = padded_mut0[(pos - 1):(pos + 512)]
    print(mut0_emb.shape)

TypeError: only integer tensors of a single element can be converted to an index